# Lesson 2: Cleaning Data
The goal for this week is to get your data set ready for exploratory analysis. This notebook will step you through some common issues you may encounter in loading your data set, using a subset of the AgCensus 2012 data set. 

The [Quick Stats API GUI interface](http://quickstats.nass.usda.gov/?source_desc=CENSUS), which may be useful in seeing how different variables are connected. This data set has many tables merged together in a way that makes it difficult to use. Also see: [methodology](http://www.agcensus.usda.gov/Publications/2012/Full_Report/Volume_1,_Chapter_1_US/usappxa.pdf) and [full report](http://www.agcensus.usda.gov/Publications/2012/Full_Report/Volume_1,_Chapter_1_US/usv1.pdf).

In [ ]:
import pandas
import os

## Read in data

### Missing data codes
When you read data into a pandas DataFrame, specify the values for missing data codes. The AgCensus data uses various  codes for missing data, depending on the reason it is missing: '(D)' means that the data point was redacted for privacy reasons because the producer could be uniquely identified, '(H)' indicates a large error as percentage of the mean.

### Date/datetime parser
Specify columns that are dates or datetimes in the argument parse_dates.

In [ ]:
datafilename = os.path.join("data", "AgCensus2012_Subset.txt")
delimiter = '\t'
missingvalues = ['(D)']
agcensus = pandas.read_csv(datafilename, sep=delimiter, na_values=missingvalues, parse_dates=[36])

In [ ]:
# View the column headers. Note that columns is an attribute of the dataframe.
agcensus.columns

### Examine top, bottom, and summary of data.

In [ ]:
help(agcensus.head)

In [ ]:
# Call the function head to veiw the first five rows. 
agcensus.head()

In [ ]:
# Optionally, we can pass an integer as an argument to change the number of rows displayed. 
# For more information, view the documentation for any command, class, or module by issuing the command help(name)
help(agcensus.head)

In [ ]:
agcensus.tail()

In [ ]:
agcensus.describe()

### View a slice of the data frame 

In [ ]:
agcensus[20:25]

### View a column of the dataframe
Note there are two ways of referring to the the column, using a "." to scope the name, or using the name as a key value in square brackets.

In [ ]:
agcensus.YEAR

In [ ]:
agcensus["YEAR"]

### View more information about the dataframe

In [ ]:
agcensus.info()

In [ ]:
agcensus.dtypes

## Rename variables
Before we continue with cleaning up the data, let's change the column names to remove the special character % and make all words lowercase.

In [ ]:
agcensus.columns = [x.lower().replace('%', 'percentage') for x in agcensus.columns]
# Verify that this had the expected results
agcensus.dtypes

## Type casting
Note that most columns are of the 'object' type, even ones that we expect to be numbers. This will happen if there are mixed types or if the numbers are reported with commas. To cast a column to a numerical type, we first need to remove any non-numerical values, then apply a string to number casting function.

For the first example, we will convert the value column to a integers. The documentation notes that values with less than half the unit of measure are reported as '(Z)'. Before we can cast the value into integers, we must remove these string values. Since they represent a number below one-half, let's round down to zero.

In [ ]:
import locale
# The 'inplace' argument makes the change in the agcensus dataframe, 
# rather than returning a copy with the replacement made.
agcensus.value.replace('(Z)', 0, inplace=True)

# Now we should be able to cast the column into a number type. To do this we need the locale module, and 
# before using it, the locale must be set so it knows what format the numbers will be in (commas representing 
# thousands separator).
locale.setlocale(locale.LC_NUMERIC, '')
# To convert a string to an integer, ues locale.atoi(), which converts from a string like "1,234" to an integer 1234.
# Similarly, locale.atof() converts from string to float.
# The apply function in pandas will apply a passed function to all values in the series. Note that it returns 
# a copy of the series, so we have to assign the original column name to refer to the new copy.
# The lambda function is necessary because NaN values have type float, and locale.atof can only handle strings.
agcensus.value = agcensus.value.apply(lambda v: locale.atof(v) if isinstance(v, str) else v)

In [ ]:
agcensus.dtypes

## Check for duplicates

In [ ]:
sum(agcensus.duplicated())

## Separate table by kind of variable
This particular data set has a lot of different data tables merged together in a way that is difficult to work with. Here we will separate out just a table about farm sizes.

In [ ]:
# first define some filters

# the word "FARMS" should appear in the group description
mask_farm = agcensus["group_desc"].apply(lambda v: "FARMS" in v) 

# the statistical category description should be "AREA"
mask_area = agcensus["statisticcat_desc"] == "AREA"

# the unit description should be "ACRES"
mask_acres = agcensus["unit_desc"] == "ACRES"

# Now use the filters to mask the rows in agcensus dataframe, returning only those that 
# are "True" for the three filters above.
farmsizes = agcensus[mask_farm & mask_area & mask_acres]

# view the resulting dataframe
farmsizes

In [ ]:
# view unique commodity descriptions
set(farmsizes.commodity_desc)